# Template extraction #

Parse text documents which are based on a template and stored in an excel document

Install dependencies and import the nlp_datau package

In [ ]:
!pip install -r ./../requirements.txt
import sys
sys.path.append("./../")


## 1. Extract substring of substring ##

Set variables

In [ ]:
path_excel_sheet_in = '/home/jovyan/work/E/XML/19/report.xlsx'
path_excel_sheet_out = '/home/jovyan/work/E/XML/19/report+nevenbevindingen.xlsx'

column_source = 'text'
column_target = 'conclusie_bevindingen'

regex_conclusion = r"Conclusie:(.*?)(DISCLAIMER|Met collegiale groet)"
regex_incidental_findings = r"(?<=Nevenbevindingen:)(.*?)(?=(DISCLAIMER|Met collegiale groet))"

import pandas
from IPython.core.display import display
df = pandas.read_excel(path_excel_sheet_in, header=0)
display(df)

Run and write to out file

In [ ]:
from nlp_datau.string_utils import select_regex


df[column_target] = df[column_source].apply(lambda x:select_regex(x, regex=regex_conclusion))
df[column_target] = df[column_target].apply(lambda x:select_regex(x, regex=regex_incidental_findings))

display(df)
df.to_excel(path_excel_sheet_out)